# NOTE:
The key different between agent as tool (.as_tool method use in the previous lab) is that:
  - the execution flow is hand from one agent to another, the other agent take over the workflow
  - whereas agent as tool call another agent, and continue the flow

# LESSON:
hand off require explicit what data to be sent, here is except from the answer sheet:
```
Pass ONLY the winning email draft to the 'Email Manager' agent
```
and
```
must hand off exactly ONE email to the Email Manager — never more than one.
```

Without these explicit instruction, the email agent may be hand offed without any data to run.



In [1]:
from dotenv import load_dotenv

from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent

import os
import asyncio
from typing import Dict


load_dotenv(override=True)
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
from  trd_agent import ChatOllama, ChatDeepSeek

llm_deepseek = ChatDeepSeek(model="deepseek-chat", model_tag="deepseek", temperature=0)

In [3]:
instruction1 = "you are a sales agent working for ComplAI, \
a company providing SaaS for ensuring SOC2 compliance, and preparing for audit \
power y AI. You write professional, serios, cold email"

instruction2 = "you are a humorous, engaging sales agent working for ComlAI, \
a company providing SaaS for ensuring SOC2 compliance, and preparing for audit \
You write witty, engaging cold emails that are likely to get a response"

instruction3 = "you are a busy sales agent working for ComlAI, \
a company providing SaaS for ensuring SOC2 compliance, and preparing for audit \
You write concise, to the point cold email."

sales_agent1 = Agent(name="Professional Sales Agent", instructions=instruction1, model="gpt-4o-mini")
sales_agent2 = Agent(name="Professional Sales Agent", instructions=instruction2, model="gpt-4o-mini")
sales_agent3 = Agent(name="Professional Sales Agent", instructions=instruction3, model="gpt-4o-mini")

tool_sale1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")
tool_sale2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description="Write a cold sales email")
tool_sale3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description="Write a cold sales email")

@function_tool
def send_email(body: str):
    print ("pretending to send an email")
    print ("sending....")
    print (f"\n\n#############{body}###########\n\n")
    print ("done!")
    return {"status": "success"}

writing_tools = [tool_sale1, tool_sale2, tool_sale3]

writing_instruction = """You are a sale manager working for ComlAI. Use the tools given to generate cold sale emails.
Never generate sale emails by yourself; alway use tools.
Try all 3 sale agent tools, each tool only once.
Pick a single best email from your own judment of which email will be most effective.
After picking the best email, hand off to the Email Manager."""

writing_instruction = """You are a sale manager working for ComlAI. Use the tools given to generate cold sale emails.
Never generate sale emails by yourself; always use tools.
Try all 3 sale agent tools, each tool only once.
Pick a single best email from your own judgment of which email will be most effective.
NOTE: the best email MUST be enclose by [BEGIN BEST EMAIL] and [END BEST EMAIL] tags
After choosing the best email, hand off to Email Manager:
"""


In [4]:
subject_instruction = "Given a message, you write a subject for a cold sales email that is likely to get a response"

html_instruction = "Given a text email body which may have some markdown, convert it to an HTML email with simple, clear, compeling layout and design"


subject_writer = Agent(name="Email subject writer", instructions=subject_instruction, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name = "subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instruction, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter", tool_description="Convert a text email body to an HTML email body")

@function_tool
def send_html_email(subject: str, html_body: str)->Dict[str,str]:
    """ Send out an email with given subject and HTML body to all sales prospect """
    print ("pretending to send an html email")
    print ("sending....")
    print (f"\n\n#############{subject}###########\n\n")
    print (f"\n\n#############{html_body}###########\n\n")
    print ("done!")
    return {"status": "success"}

email_tools = [subject_tool, html_tool, send_html_email]

email_tools


[FunctionTool(name='subject_writer', description='Write a subject for a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'subject_writer_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7d4c2b51b250>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='html_converter', description='Convert a text email body to an HTML email body', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'html_converter_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7d4c2b51b5b0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 Fu

In [6]:
email_instruction="""You are email formatter and sender.
Given body of an email to be sent, you first use the subject_writer tool to write subject for the email.
Then, use the html_converter tool to convert to body to HTML.
Finally, use send_html_email tool to send out the email with subject and HTML body."""
emailer_agent = Agent(name="Email Manager", instructions=email_instruction, tools=email_tools, model="gpt-4o-mini", handoff_description="Convert an email to HTML and send it")



# NOTE: This is the key step to prune hand of context
# Only previous query and agent text output will be passed

In [8]:
from agents import handoff
from agents.extensions import handoff_filters
email_handoff = handoff(
    agent=emailer_agent,
    input_filter=handoff_filters.remove_all_tools,  # <--- replaces your custom filter
)

In [9]:
sales_manager = Agent(
    name="Sale Manager",
    instructions=writing_instruction,
    model="gpt-4o-mini",
    tools=writing_tools,
    handoffs=[email_handoff],
)
message = "Send a cold sale email addressed to 'Dear CEO'"

In [10]:
print (sales_manager)

Agent(name='Sale Manager', handoff_description=None, tools=[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7d4c2b51a5f0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None), FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7d4c2b51a950>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrai

In [11]:
with trace("Automated SDR handoff"):
    result = await Runner.run(sales_manager, message)

pretending to send an html email
sending....


#############Transform Compliance: Simplified Like Your Favorite Takeout!###########




#############<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Email to CEO</title>
    <style>
        body {
            font-family: Arial, sans-serif;
            color: #333;
            background-color: #f9f9f9;
            padding: 20px;
        }
        .container {
            max-width: 600px;
            margin: 0 auto;
            background: #ffffff;
            border-radius: 8px;
            box-shadow: 0 2px 5px rgba(0, 0, 0, 0.1);
            padding: 20px;
        }
        h1 {
            font-size: 24px;
            color: #2c3e50;
        }
        p {
            font-size: 16px;
            line-height: 1.5;
        }
        .footer {
            font-size: 14px;
            color: #aaa;
            margin-top: 20px;
       

In [12]:
print (result.final_output)

The cold sales email has been successfully sent to the CEO. If you need anything else or want to send more emails, just let me know!


In [38]:
#!jupyter nbconvert --to python lab3_agent_handoff_pruned_context.ipynb

[NbConvertApp] Converting notebook lab3_agent_handoff_pruned_context.ipynb to python
[NbConvertApp] Writing 7938 bytes to lab3_agent_handoff_pruned_context.py
